# Today's topic: How to set your configs right: Open Cost In Bytes

# 0. Set-Ups

General hints for this notebook:
- Spark UI usually accesible by http://localhost:4040/ or http://localhost:4041/
- Deep dive Spark UI happens in later episodes
- sc.setJobDescription("Description") replaces the Job Description of an action in the Spark UI with your own
- sdf.rdd.getNumPartitions() returns the number partitions of the current Spark DataFrame
- sdf.write.format("noop").mode("overwrite").save() is a good way to analyze and initiate actions for transformations without side effects during an actual write

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import gresearch.spark.parquet
import math
import time
import pandas as pd
import os
import sys


In [2]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
spark = SparkSession \
    .builder \
    .appName("Data with Nikk the Greek Spark Session") \
    .master("local[4]") \
    .enableHiveSupport() \
    .config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.5") \
    .getOrCreate()

"""
Reference gresearch:
- Parquet files analysis: https://www.gresearch.com/blog/article/parquet-files-know-your-scaling-limits/
- GitHub Spark extension: https://github.com/G-Research/spark-extension
- Parquet methods: https://github.com/G-Research/spark-extension/tree/master/python/gresearch/spark/parquet
"""
sc = spark.sparkContext

In [4]:
#Turning off AQE as it generates more jobs which might be confusing for this scenario here. 
spark.conf.set("spark.sql.adaptive.enabled", "false")
#to not cache datafrimes... this may not create repeatable results
spark.conf.set("spark.databricks.io.cache.enabled", "false")

In [5]:
def sdf_generator(num_rows: int, num_partitions: int = None) -> "DataFrame":
    return (
        spark.range(num_rows, numPartitions=num_partitions)
        .withColumn("date", f.current_date())
        .withColumn("timestamp",f.current_timestamp())
        .withColumn("idstring", f.col("id").cast("string"))
        .withColumn("idfirst", f.col("idstring").substr(0,1))
        .withColumn("idlast", f.col("idstring").substr(-1,1))
        )

In [6]:
def rows_per_partition(sdf: "DataFrame") -> None:
    num_rows = sdf.count()
    sdf_part = sdf.withColumn("partition_id", f.spark_partition_id())
    sdf_part_count = sdf_part.groupBy("partition_id").count()
    sdf_part_count = sdf_part_count.withColumn("count_perc", 100*f.col("count")/num_rows)
    sdf_part_count.orderBy("partition_id").show()

In [7]:
def rows_per_partition_col(sdf: "DataFrame", num_rows: int, col: str) -> None:
    sdf_part = sdf.withColumn("partition_id", f.spark_partition_id())
    sdf_part_count = sdf_part.groupBy("partition_id", col).count()
    sdf_part_count = sdf_part_count.withColumn("count_perc", 100*f.col("count")/num_rows)
    sdf_part_count.orderBy("partition_id", col).show()


In [8]:
BASE_DIR = "D:/Spark/Data"

In [9]:
def write_generator(num_rows, num_files):
    sdf = sdf_generator(num_rows, num_files)
    path = f"{BASE_DIR}/{num_files}_files_{num_rows}_rows.parquet"
    sc.setJobDescription(f"Write {num_files} files, {num_rows} rows")
    sdf.write.format("parquet").mode("overwrite").save(path)
    sc.setJobDescription("None")
    print(f"Num partitions written: {sdf.rdd.getNumPartitions()}")
    print(f"Saved Path: {path}")
    return path

In [10]:
def set_configs(maxPartitionsMB = 128, openCostInMB = 4, minPartitions = 4):
    maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
    openCostInBytes = math.ceil(openCostInMB*1024*1024)
    spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")
    spark.conf.set("spark.sql.files.openCostInBytes", str(openCostInBytes)+"b")
    spark.conf.set("spark.sql.files.minPartitionNum", str(minPartitions))
    print(" ")
    print("******** SPARK CONFIGURATIONS ********")
    print(f"MaxPartitionSize {maxPartitionsMB} MB or {maxPartitionsBytes} bytes")
    print(f"OpenCostInBytes {openCostInMB} MB or {openCostInBytes} bytes")
    print(f"Min Partitions: {minPartitions}")

In [11]:
def get_parquet_meta_data(path):
    sdf = (
        spark.read.parquet_metadata(path)
        .select("filename", "blocks", "compressedBytes", "rows")
        .dropDuplicates(["filename"])
        .withColumn("compressedMB", f.round(f.col("compressedBytes")/1024/1024, 1))
        .withColumn("calcNumBlocks", f.col("compressedMB")/128)
    )
    sdf.show(20, truncate=False)
    return sdf

In [12]:
def get_parquet_blocks(path):
    sdf = (
        spark.read.parquet_blocks(path)
        .dropDuplicates(["filename","block"])
        .orderBy("filename", "block")
        .withColumn("blockEnd", f.col("blockStart") + f.col("compressedBytes") - 1)
        .withColumn("blockMiddle", f.col("blockStart") + 0.5 * f.col("compressedBytes"))
        .withColumn("compressedMB", f.round(f.col("compressedBytes")/1024/1024, 1))
        .select("filename", "block", "blockStart", "blockEnd", "blockMiddle", "compressedBytes", "compressedMB", "rows")
    )

    sdf.show(20, truncate=False)

In [13]:
def get_spark_partitions(path):
    sdf = (
        spark.read.parquet_partitions(path)
        .withColumn("compressedMB", f.round(f.col("compressedBytes")/1024/1024, 1))
        .select("partition", "start", "end", "length", "blocks", "compressedBytes", "compressedMB", "rows", "filename")
    )

    sdf.show(20, truncate=False)

In [14]:
def get_parquet_window_length(path):
    sdf = spark.read.parquet_partitions(path)
    val = sdf.select(f.max(sdf["length"]))
    max_length = val.collect()[0][0]
    print(f"Max Parquet window length: {round(max_length/1024/1024, 1)} MB or {max_length} bytes")

In [15]:
def get_parquet_file_size(path):
    sdf = (
        spark.read.parquet_metadata(path)
        .select("filename", "blocks", "compressedBytes", "rows")
        .dropDuplicates(["filename"])
    )
    sum = sdf.select(f.sum(sdf["compressedBytes"]))
    size = sum.collect()[0][0]
    return size

In [16]:
def bytes_rows_per_partition(path):
    sdf = (
        spark.read.parquet_partitions(path)
        .groupBy("partition").agg(f.sum("compressedBytes"), f.sum("rows"), f.count("partition"))
        .withColumnRenamed("sum(compressedBytes)", "compressedBytes")
        .withColumnRenamed("sum(rows)", "rows")
        .withColumnRenamed("count(partition)", "numFiles")
        .withColumn("compressedMB", f.round(f.col("compressedBytes")/1024/1024, 1))
        .select("partition", "numFiles", "compressedBytes","compressedMB","rows")
        .orderBy("partition")
    )
    sdf.show(20)
    return sdf

In [17]:
def file_analysis(path, num_files):
    file_size = get_parquet_file_size(path)
    avg_file_size = file_size/num_files
    print(" ")
    print("******** FILE SIZE ANALYSIS ********")
    print(f"File Size: {round(file_size/1024/1024, 1)} MB or {file_size} bytes")
    print(f"Num files: {num_files}")
    print(f"Avg file Size: {round(avg_file_size/1024/1024, 1)} MB or {avg_file_size} bytes")
    


In [18]:
def row_count_analysis(num_files, num_rows):
    print(" ")
    print("******** ROW COUNT ANALYSIS ********")    
    print(f"Num files written: {num_files}")
    print(f"Num rows written: {num_rows}")
    print(f"Num rows per file: {int(num_rows/num_files)}")

In [19]:
def get_actual_num_partitions(path):
    sdf = spark.read.parquet(path)
    print(" ")
    print("******** ACTUAL RESULTS ********")   
    print(f"ActualNumPartitions: {sdf.rdd.getNumPartitions()}")
    return sdf.rdd.getNumPartitions()


In [20]:
def noop_write(path, desc = ""):
    sdf = spark.read.parquet(path)
    sc.setJobDescription(f"WRITE {desc}")
    start_time = time.time()
    sdf.write.format("noop").mode("overwrite").save()
    end_time = time.time()
    sc.setJobDescription("None")
    duration = round(end_time - start_time, 2)
    print(f"Duration: {duration} sec")
    return duration

In [21]:
def maxSplitBytes(file_size, num_files):
    """
    Reference to code: 
    - Stackoverflow: https://stackoverflow.com/questions/70985235/what-is-opencostinbytes
    - GitHub: https://github.com/apache/spark/blob/v3.3.1/sql/core/src/main/scala/org/apache/spark/sql/execution/datasources/FilePartition.scala#L86-L97
    """
    maxPartitionBytes = int(spark.conf.get("spark.sql.files.maxPartitionBytes")[:-1])
    openCostInBytes = int(spark.conf.get("spark.sql.files.openCostInBytes")[:-1])
    minPartitionNum = int(spark.conf.get("spark.sql.files.minPartitionNum"))
    
    file_size_padded = file_size + num_files * openCostInBytes
    size_per_core_padded = file_size_padded / minPartitionNum
    max_partition_size = int(min(maxPartitionBytes, max(openCostInBytes, size_per_core_padded)))
    no_partitions_padded = file_size_padded/max_partition_size

    print(" ")
    print("******** ADVANCED ALGORITHM TO ESTIMATE Partition SIZE AND NO PARTITIONS ********")
    print(f"Padded File Size: {round(file_size_padded/1024/1024, 1)} MB or {file_size_padded} bytes")
    print(f"SizePerCorePadded: {round(size_per_core_padded/1024/1024, 1)} MB or {size_per_core_padded} bytes")
    print(f"MaxPartionsize: {round(max_partition_size/1024/1024, 1)} MB or {max_partition_size} bytes")
    print(f"EstimatedPartitionsAvg: {math.ceil(no_partitions_padded)}, unrounded: {no_partitions_padded}")
    return max_partition_size, size_per_core_padded

In [22]:
def get_files_as_list(parquet_meta_data_sdf):
    return list(parquet_meta_data_sdf.select("compressedBytes").toPandas()["compressedBytes"])

In [23]:
def split_files(files_list, max_split_bytes):
    """
    Reference to code in GitHub: https://github.com/apache/spark/blob/master/sql/core/src/main/scala/org/apache/spark/sql/execution/PartitionedFileUtil.scala#L45
    """
    result_list = []
    for file_size in files_list:
        remaining = file_size - max_split_bytes
        f = {
            "start": 0,
            "length": min(max_split_bytes, file_size),
            "file_size": file_size
        }
        result_list.append(f)
        while remaining > 0:
            if remaining > max_split_bytes:
                f = {
                    "start": file_size - remaining,
                    "length": max_split_bytes,
                    "file_size": 0
                }
                result_list.append(f)
            else:
                f = {
                    "start": file_size - remaining,
                    "length": remaining,
                    "file_size": 0
                }
                result_list.append(f)  
            remaining = remaining - max_split_bytes
    return sorted(result_list, key=lambda d: d['length'], reverse=True) 


In [24]:
def getFilePartitions(splitted_files_list, max_split_bytes):
    """
    Reference to code in GitHub: https://github.com/apache/spark/blob/v3.3.1/sql/core/src/main/scala/org/apache/spark/sql/execution/datasources/FilePartition.scala
    """
    openCostInBytes = int(spark.conf.get("spark.sql.files.openCostInBytes")[:-1])
    partitions = []
    current_files = []
    current_size = 0

    def close_partition():
        if current_files:
            partition_details = {
                "files": current_files.copy(),
                "num_files": len(current_files),
            }
        else:
            partition_details = {}
        partitions.append(partition_details)
        current_files.clear()

    for file in splitted_files_list:
        if current_size + file["length"] > max_split_bytes:
            close_partition()
            current_size = 0
        current_size += file["length"] + openCostInBytes
        current_files.append(file)
    close_partition()
    print(f"Number calculated Partitions: {len(partitions)}")
    return partitions


In [25]:
def file_partitions_analysis(file_partitions):
    pdf = pd.DataFrame(file_partitions)
    pdf["partition"] = pdf.index
    sdf_partitions = spark.createDataFrame(pdf)
    sdf_partitions = sdf_partitions.withColumn("files", f.explode(sdf_partitions.files))
    sdf_partitions = (sdf_partitions
                    .withColumn("start", f.col("files").start)
                    .withColumn("length", f.col("files").length)
                    .withColumn("file_size", f.col("files").file_size)
                    .drop("files", "num_files")
                    .withColumn("virt_num_files", f.lit(1))
                    .withColumn("real_num_files", f.when(f.col("file_size") > 0, f.lit(1)).otherwise(f.lit(0)))
                    .select("partition", "start", "length", "file_size", "real_num_files", "virt_num_files")
    )
    #sdf_partitions.show()
    sdf_agg = sdf_partitions.groupBy("partition").agg(f.sum("file_size")).withColumnRenamed("sum(file_size)", "file_size")
    sdf_agg = sdf_agg.orderBy("partition")
    sdf_agg = sdf_agg.withColumn("file_size_mb", f.col("file_size")/1024/1024)
    #sdf_agg.show()
    max_size = sdf_agg.select(f.max(sdf_agg["file_size"])).collect()[0][0]
    min_size = sdf_agg.select(f.min(sdf_agg["file_size"])).collect()[0][0]
    median_size = sdf_agg.select(f.median(sdf_agg["file_size"])).collect()[0][0]
    sdf_empty = sdf_agg.filter(f.col("file_size") == 0)
    empty_partitions = sdf_empty.count()
    return {
        "max_size": max_size,
        "min_size": min_size,
        "median_size": median_size,
        "num_partitions": len(file_partitions),
        "empty_partition": empty_partitions
    }
    

# 1 - Max Cost Per Bytes
  - Represents the cost of creating a new partition
  - based on the config "spark.sql.files.openCostInBytes"
  - defaults to 4 MB
  - Technically it adds the cost, e.g. 4 MB, to each file which is called padding
  - Through this less but bigger partitions are created around the size of the open cost value
  - Usually no influence, except of smaller files, default of 4MB works
  - Official description: The estimated cost to open a file, measured by the number of bytes that could be scanned in the same time. This is used when putting multiple files into a partition. It is better to over-estimate, then the partitions with small files will be faster than partitions with bigger files (which is scheduled first). This configuration is effective only when using file-based sources such as Parquet, JSON and ORC.

References:
- https://stackoverflow.com/questions/70985235/what-is-opencostinbytes
- https://stackoverflow.com/questions/69034543/number-of-tasks-while-reading-hdfs-in-spark
- https://stackoverflow.com/questions/75924368/skewed-partitions-when-setting-spark-sql-files-maxpartitionbytes
- https://spark.apache.org/docs/latest/sql-performance-tuning.html
- https://www.linkedin.com/pulse/how-initial-number-partitions-determined-pyspark-sugumar-srinivasan#:~:text=Ideally%20partitions%20will%20be%20created,resource%20will%20get%20utilised%20properly

# 2 - Small Dataset

In [167]:
num_files = 4500
num_rows = 256000000
path = write_generator(num_rows, num_files)

Num partitions written: 4500
Saved Path: D:/Spark/Data/4500_files_256000000_rows.parquet


In [26]:
path = "D:/Spark/Data/4500_files_256000000_rows.parquet"
num_files = 4500
num_rows = 256000000
sdf_meta_data = get_parquet_meta_data(path)
file_analysis(path, num_files)
row_count_analysis(num_files, num_rows)
file_size = get_parquet_file_size(path)
file_size_list = get_files_as_list(sdf_meta_data)


+-------------------------------------------------------------------------------------------------------------------------+------+---------------+-----+------------+-------------+
|filename                                                                                                                 |blocks|compressedBytes|rows |compressedMB|calcNumBlocks|
+-------------------------------------------------------------------------------------------------------------------------+------+---------------+-----+------------+-------------+
|file:/D:/Spark/Data/4500_files_256000000_rows.parquet/part-04275-b0e09747-0806-42af-a3e5-3397ed107c14-c000.snappy.parquet|1     |509992         |56888|0.5         |0.00390625   |
|file:/D:/Spark/Data/4500_files_256000000_rows.parquet/part-01861-b0e09747-0806-42af-a3e5-3397ed107c14-c000.snappy.parquet|1     |509449         |56889|0.5         |0.00390625   |
|file:/D:/Spark/Data/4500_files_256000000_rows.parquet/part-03992-b0e09747-0806-42af-a3e5-3397ed107c

# 3 - Open Costs

In [27]:
results = []
results_dict = {}
for openCostMB in [20, 10, 8, 5, 4, 3, 2, 1, 0.5, 0.25, 0.1, 0.01, 0.001, 0]:
    maxMB = 128
    minPartitions = 4
    set_configs(maxPartitionsMB=maxMB, openCostInMB=openCostMB, minPartitions=minPartitions)
    max_split_bytes, size_per_core_padded = maxSplitBytes(file_size, num_files)
    splitted_files = split_files(file_size_list, max_split_bytes)
    file_partitions = getFilePartitions(splitted_files, max_split_bytes)
    results_dict = file_partitions_analysis(file_partitions)
    results_dict["maxPartitionMB"] = maxMB
    results_dict["openCosts"] = openCostMB
    results_dict["size_per_core_padded"] = round(size_per_core_padded/1024/1024,1)
    results_dict["maxSplitBytes"] = round(max_split_bytes/1024/1024,1)
    results_dict["actualNumberPartitions"] = get_actual_num_partitions(path)
    results_dict["ExecutionTime"] = noop_write(path, str(maxMB))
    #bytes_rows_per_partition(path)
    results.append(results_dict)

 
******** SPARK CONFIGURATIONS ********
MaxPartitionSize 128 MB or 134217728 bytes
OpenCostInBytes 20 MB or 20971520 bytes
Min Partitions: 4
 
******** ADVANCED ALGORITHM TO ESTIMATE Partition SIZE AND NO PARTITIONS ********
Padded File Size: 92130.1 MB or 96605364433 bytes
SizePerCorePadded: 23032.5 MB or 24151341108.25 bytes
MaxPartionsize: 128.0 MB or 134217728 bytes
EstimatedPartitionsAvg: 720, unrounded: 719.7660538032651
Number calculated Partitions: 643
 
******** ACTUAL RESULTS ********
ActualNumPartitions: 643
Duration: 31.99 sec
 
******** SPARK CONFIGURATIONS ********
MaxPartitionSize 128 MB or 134217728 bytes
OpenCostInBytes 10 MB or 10485760 bytes
Min Partitions: 4
 
******** ADVANCED ALGORITHM TO ESTIMATE Partition SIZE AND NO PARTITIONS ********
Padded File Size: 47130.1 MB or 49419444433 bytes
SizePerCorePadded: 11782.5 MB or 12354861108.25 bytes
MaxPartionsize: 128.0 MB or 134217728 bytes
EstimatedPartitionsAvg: 369, unrounded: 368.2035538032651
Number calculated Part

In [28]:
pdf = pd.DataFrame(results)
sdf_results = spark.createDataFrame(pdf)
sdf_results = sdf_results.withColumn("min_size", f.round(f.col("min_size")/1024/1024,1))
sdf_results = sdf_results.withColumn("median_size", f.round(f.col("median_size")/1024/1024,1))
sdf_results = sdf_results.withColumn("max_size", f.round(f.col("max_size")/1024/1024,1))
sdf_results = sdf_results.select("maxPartitionMB", "openCosts", "size_per_core_padded", "maxSplitBytes", "num_partitions", "actualNumberPartitions", "empty_partition", "min_size", "median_size", "max_size", "ExecutionTime")
sdf_results.show(100)

+--------------+---------+--------------------+-------------+--------------+----------------------+---------------+--------+-----------+--------+-------------+
|maxPartitionMB|openCosts|size_per_core_padded|maxSplitBytes|num_partitions|actualNumberPartitions|empty_partition|min_size|median_size|max_size|ExecutionTime|
+--------------+---------+--------------------+-------------+--------------+----------------------+---------------+--------+-----------+--------+-------------+
|           128|       20|             23032.5|        128.0|           643|                   643|              0|     2.7|        3.4|     3.4|        31.99|
|           128|       10|             11782.5|        128.0|           347|                   347|              0|     0.9|        6.3|     6.3|        27.57|
|           128|        8|              9532.5|        128.0|           282|                   282|              0|     1.8|        7.7|     7.8|        28.65|
|           128|        5|              

In [29]:
save_path = f"D:/Spark/Data/ResultsOpenCosts/open3.parquet"
sdf_results.write.format("parquet").save(save_path)

In [44]:
load_path = f"D:/Spark/Data/ResultsOpenCosts/open3.parquet"
sdf = spark.read.parquet(load_path)
sdf.orderBy(f.col("openCosts"), ascending=False).show()

+--------------+---------+--------------------+-------------+--------------+----------------------+---------------+--------+-----------+--------+-------------+
|maxPartitionMB|openCosts|size_per_core_padded|maxSplitBytes|num_partitions|actualNumberPartitions|empty_partition|min_size|median_size|max_size|ExecutionTime|
+--------------+---------+--------------------+-------------+--------------+----------------------+---------------+--------+-----------+--------+-------------+
|           128|     20.0|             23032.5|        128.0|           643|                   643|              0|     2.7|        3.4|     3.4|        31.99|
|           128|     10.0|             11782.5|        128.0|           347|                   347|              0|     0.9|        6.3|     6.3|        27.57|
|           128|      8.0|              9532.5|        128.0|           282|                   282|              0|     1.8|        7.7|     7.8|        28.65|
|           128|      5.0|              

In [3]:
4500/643

6.998444790046657